In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
import time
import re
import json
#ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
driver = webdriver.Chrome()
driver.maximize_window()
url = 'https://worldwide.espacenet.com/'
search_key = '((ctxt all "E-commerce" OR ctxt all "ecommerce") OR ctxt all "smart retail*" OR ctxt all "future retail*") AND ((ctxt all "etransaction" AND ctxt all "e-transaction") OR (ctxt all "mobile commerce" AND ctxt all "m-commerce") OR ctxt all "e-banking" OR ctxt all "e-crm" OR ctxt all "crypto-pay*" OR nftxt all "ict" OR ctxt all "e-auction" OR ctxt all "THIRD PARTY PAY*" OR ctxt all "NFC" OR ctxt all "Mobile Wallet*" OR ctxt all "e-micropayment*")'
time_sleep = 2
implicitly = 20
#số lượng items cần cào
quantity = 20
final_data = list()

In [3]:
def get_site(url):
    driver.get(url)
    driver.implicitly_wait(implicitly)

In [4]:
def search_by_keyword():
    #get_btn_search
    btn_search= driver.find_element_by_xpath("//*[starts-with(@class, 'search__input')]")
    #send key search into input btn_search
    btn_search.send_keys(search_key)
    #search
    btn_search.send_keys(Keys.ENTER);
    #pause to load
    time.sleep(time_sleep)

In [5]:
def fix_page_search():
    driver.execute_script("document.querySelector('#search-page-row > div').style.width='400px';")

In [6]:
def get_num(s):
        return int((re.search(r'\d+', s)[0]))

In [7]:
def clean_text(s):
    return re.sub('·', ' ', re.sub('\s+', ' ', s)) 

In [8]:
def crwal_by_id(_id):
    return clean_text(driver.find_element_by_id(_id).text)

In [9]:
#event click claims and biblio
def to_claims():
    driver.find_element_by_xpath("//span[text()='Claims']").click()
def to_biblio():
    driver.find_element_by_xpath("//span[text()='Bibliographic data']").click()

In [10]:
def main_crawl_biblio():
    Applicants = crwal_by_id('biblio-applicants-content')
    Inventors = crwal_by_id('biblio-inventors-content')
    Application = crwal_by_id('biblio-application-number-content')
    Publication = crwal_by_id('biblio-publication-number-content')
    IPC = crwal_by_id('biblio-publication-number-content')
    Published_as = crwal_by_id('biblio-also-published-as-content')
    Title = crwal_by_id('biblio-title-content')
    Abstract= crwal_by_id('biblio-abstract-content')
    
    return {
        'Applicants':Applicants,
        'Inventors':Inventors,
        'Application':Application,
        'Publication':Publication,
        'IPC':IPC,
        'Published as':Published_as,
        'Title':Title,
        'Abstract':Abstract
    }

In [11]:
def main_crawl():
    driver.implicitly_wait(implicitly)
#     try:
    data = main_crawl_biblio()
#     except:
#         data = {}
    driver.implicitly_wait(implicitly)
    to_claims()
    driver.implicitly_wait(implicitly)
    try:
        data['Claims'] = clean_text(driver.find_element_by_xpath("//div[starts-with(@class, 'text-block__content')]").text)
    except:
        data['Claims'] = None
    driver.implicitly_wait(implicitly)
    to_biblio()
    return data

In [12]:
def scroll_items_search():
    def scroll_publications_list():
        element = driver.find_element_by_xpath("//*[starts-with(@class, 'publications-list--')]")
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', element)
    while (quantity  > get_num(driver.find_elements_by_xpath("//*[starts-with(@class, 'item__section--')]/header")[-1].text)):
        scroll_publications_list()

In [13]:
def crwal_data():
    items = driver.find_elements_by_xpath("//*[starts-with(@class, 'item__section')]")
    idx =0
    for item in items:
        idx +=1
        try:
            item.click()
            driver.implicitly_wait(implicitly)
            final_data.append(main_crawl())
            print(f"crawl: {idx}")
        except:
            to_biblio()
            print(f"crawl-error: {idx}")
            continue

In [14]:
def to_json():
    with open('data.json', 'w') as f:
        json.dump(final_data, f, indent=4)

In [15]:
def run():
    get_site(url)
    search_by_keyword()
    fix_page_search()
    scroll_items_search()
    crwal_data()
    to_json()

In [16]:
run()

crawl: 1
crawl: 2
crawl: 3
crawl: 4
crawl: 5
crawl: 6
crawl: 7
crawl: 8
crawl: 9
crawl: 10
crawl: 11
crawl: 12
crawl: 13
crawl: 14
crawl: 15
crawl: 16
crawl: 17
crawl: 18
crawl: 19
crawl: 20
